<a href="https://colab.research.google.com/github/SELF-msselve/UTN/blob/main/CEL_Carga_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga a una base de datos
Versión simplificada


## Librerías a utilizar
- sqlalchemy (Versión **menor** 2.0)
- psycopg2-binary (Si vamos a trabajar con PostgreSQL)
- pandas

In [ ]:
# Asegurarse de instalar una version de sqlalchemy menor a la 2.0.0
# Puede ser 1.4.x
!pip install sqlalchemy
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.9 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [ ]:
# Definimos el string de conexión
host = "...aivencloud.com"
port = 15191
database = "..."
username = "avnadmin"
password = "..."

conn_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"

In [ ]:
# Creamos la conexión y establecemos la conexión a la db
engine = create_engine(conn_string)

In [ ]:
# Se recomienda crear la tabla antes de insertar los datos
# para definir un esquema adecuado

query = text("""
-- Creamos una tabla sobre datos de cafe
CREATE TABLE IF NOT EXISTS public.coffee (
    auto_id SERIAL PRIMARY KEY, -- ID autoincremental
    coffe_id INT, -- ID del cafe en la fuente de datos original
    tittle VARCHAR(100),
    description VARCHAR(500),
    ingredients VARCHAR(500)
);
""")

# Ejecutamos la query
with engine.connect() as conn, conn.begin():
    conn.execute(query)

In [ ]:
# Obtenemos datos de una API
coffe_url = "https://api.sampleapis.com/coffee/hot"
coffe_data = requests.get(coffe_url).json()
df_coffe = pd.json_normalize(coffe_data)

# Seleccionamos algunas columnas
df_coffe = df_coffe[["id", "title", "description", "ingredients"]]

# Renombramos columnas para que coincidan con la tabla de la base de datos
df_coffe.columns = ["coffe_id", "tittle", "description", "ingredients"]

# Procesamos la columna ingredients
df_coffe["ingredients"] = df_coffe["ingredients"].apply(lambda x: ", ".join(x))

df_coffe.head()

,coffe_id,tittle,description,ingredients
0,1,Black Coffee,Svart kaffe är så enkelt som det kan bli med m...,Coffee
1,2,Latte,Som den mest populära kaffedrycken där ute bes...,"Espresso, Ångad mjölk"
2,3,Caramel Latte,Om du gillar latte med en speciell smak kan ka...,"Espresso, Ångad mjölk, Karamellsirap"
3,4,Cappuccino,Cappuccino är en latte som är gjord med mer sk...,"Espresso, Ångad mjölk, Foam"
4,5,Americano,Med en liknande smak som svart kaffe består am...,"Espresso, Hett vatten"


In [ ]:
# Guardamos los datos en la base de datos, en la tabla creada anteriormente

with engine.connect() as conn, conn.begin():
    df_coffe.to_sql(
        "coffee",
        schema="public",
        con=conn,
        if_exists="append",
        method="multi",
        index=False
    )

In [ ]:
# A modo de ejemplo, desde Pandas podemos consultar la tabla

query = text("""
SELECT *
FROM public.coffee
""")

with engine.connect() as conn, conn.begin():
    df_check = pd.read_sql(query, conn)

In [ ]:
df_check

,auto_id,coffe_id,tittle,description,ingredients
0,1,1,Black Coffee,Svart kaffe är så enkelt som det kan bli med m...,Coffee
1,2,2,Latte,Som den mest populära kaffedrycken där ute bes...,"Espresso, Ångad mjölk"
2,3,3,Caramel Latte,Om du gillar latte med en speciell smak kan ka...,"Espresso, Ångad mjölk, Karamellsirap"
3,4,4,Cappuccino,Cappuccino är en latte som är gjord med mer sk...,"Espresso, Ångad mjölk, Foam"
4,5,5,Americano,Med en liknande smak som svart kaffe består am...,"Espresso, Hett vatten"
5,6,6,Espresso,Ett espressoskott kan serveras ensamt eller an...,Espresso
6,7,7,Macchiato,Macchiaton är en annan espresso-baserad dryck ...,"Espresso, Foam"
7,8,8,Mocha,För alla chokladälskare där ute kommer ni att ...,"Espresso, Ångad mjölk, Choklad"
8,9,9,Hot Chocolate,Under kalla vinterdagar får en kopp varm chokl...,"Choklad, Mjölk"
9,10,10,Chai Latte,Om du letar efter en smakfull varm dryck mitt ...,"Te, Mjölk, Ingefära, Kardemumma, Kanel"
